In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, f1_score, classification_report
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/ptb-xl-tcc')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Carregando dados de treino e teste

In [ ]:
X_train = np.load('/content/drive/MyDrive/ptb-xl-tcc/outfile_X_train.npy', allow_pickle=True)
X_test = np.load('/content/drive/MyDrive/ptb-xl-tcc/outfile_X_test.npy', allow_pickle=True)
y_train = np.load('/content/drive/MyDrive/ptb-xl-tcc/outfile_y_train.npy', allow_pickle=True)
y_test = np.load('/content/drive/MyDrive/ptb-xl-tcc/outfile_y_test.npy', allow_pickle=True)

Função que remove elementos vazios da lista

In [ ]:
def remover(X,y):
     aux = []
     ret = 0
     x_aux = []
     y_aux = []
     for i in range(len(y)):
         if y[i] != []:
           y_aux.append(y[i])
           x_aux.append(X[i])
     return np.array(x_aux), np.array(y_aux)

In [ ]:
X_train,y_train = remover(X_train,y_train)
X_test,y_test = remover(X_test,y_test)

<ipython-input-8-c44d1cdeb708>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(x_aux), np.array(y_aux)


In [ ]:
print("X_train:" + str(X_train.shape) + "; y_train:" + str(y_train.shape) + "; X_test:" + str(X_test.shape) + "; y_test:" + str(y_test.shape))

X_train:(19237, 1000, 12); y_train:(19237,); X_test:(2159, 1000, 12); y_test:(2159,)


Função que realiza codificação multi-hot

In [ ]:
def multi_hot_encoding(y):
    #Obter dados categóricos
    y_encoded = np.array(y)

    # Criar uma instância do codificador multi-hot
    encoder = MultiLabelBinarizer()

    # Aplicar a codificação multi-hot aos dados
    y_encoded_ = encoder.fit_transform(y_encoded)

    return y_encoded_

In [ ]:
y_train = multi_hot_encoding(y_train)
y_test = multi_hot_encoding(y_test)

In [ ]:
y_train

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0]])

In [ ]:
y_train.shape

(19237, 5)

In [ ]:
y_test

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

In [ ]:
y_test.shape

(2159, 5)

Arquitetura Transformer

In [ ]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, input_dim, n_heads, ff_dim, dropout=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.attention = layers.MultiHeadAttention(
            num_heads=n_heads, key_dim=input_dim[-1] // n_heads
        )
        self.ff = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(dropout),
        ])

    def build(self, input_shape):
        #Adiciona a camada densa residual
        self.dense_residual = layers.Dense(input_shape[-1])

    def call(self, inputs):
        attention_output = self.attention(inputs, inputs)
        x = tf.keras.layers.Add()([attention_output, inputs])  #Conexão residual

        #Ajuste das dimensões usando Dense
        residual = self.dense_residual(x)
        x = tf.keras.layers.Add()([x, residual])  #Conexão residual

        x = self.ff(x)
        return x


# Definindo o modelo Transformer
class Transformer(keras.Model):
    def __init__(self, n_blocks, n_heads, ff_dim, n_classes, input_shape, rate=0.1):
        super(Transformer, self).__init__()
        self.encoder_layers = [TransformerBlock(input_shape, n_heads, ff_dim, rate) for _ in range(n_blocks)]
        self.flatten = layers.Flatten()
        self.f_connected = layers.Dense(n_classes, activation="sigmoid")

    def call(self, inputs):
        x = inputs
        for layer in self.encoder_layers:
            x = layer(x)
        x = self.flatten(x)
        x = self.f_connected(x)
        return x

In [ ]:
input_dim = X_train.shape[1:]
n_blocks = 4
n_heads = 10
ff_dim = 512
n_classes = 5

#Criar instância do modelo Transformer
model = Transformer(n_blocks=n_blocks, n_heads=n_heads, ff_dim=ff_dim, n_classes=n_classes, input_shape=input_dim, rate=0.1)

#Criar um tensor de entrada baseado nas dimensões desejadas
input_dim_ = tf.keras.Input(shape=input_dim)
model(input_dim_)  # Chamando o modelo com o tensor de entrada

#Compilar o modelo depois de chamar o modelo com o tensor de entrada
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss="binary_crossentropy",
              metrics=["accuracy",
                        tf.keras.metrics.Precision(),
                        tf.keras.metrics.Recall(),
              ])
model.summary()


Model: "transformer_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer_block_48 (Tran  multiple                  9382      
 sformerBlock)                                                   
                                                                 
 transformer_block_49 (Tran  multiple                  548382    
 sformerBlock)                                                   
                                                                 
 transformer_block_50 (Tran  multiple                  548382    
 sformerBlock)                                                   
                                                                 
 transformer_block_51 (Tran  multiple                  548382    
 sformerBlock)                                                   
                                                                 
 flatten_5 (Flatten)         multiple                

Realizando treinamento do modelo

In [ ]:
epochs = 30
batch_size = 32
early_stopping = EarlyStopping(patience=10, min_delta=1e-3, monitor="val_accuracy", restore_best_weights=True)


history = model.fit(X_train,
                    y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[early_stopping]
)

Epoch 1/30
542/542 [==============================] - 376s 673ms/step - loss: 35.3487 - accuracy: 0.2991 - precision_5: 0.3268 - recall_5: 0.3182 - val_loss: 1.1246 - val_accuracy: 0.2952 - val_precision_5: 0.3046 - val_recall_5: 0.2402
Epoch 2/30
542/542 [==============================] - 364s 671ms/step - loss: 0.7052 - accuracy: 0.3771 - precision_5: 0.4004 - recall_5: 0.2142 - val_loss: 0.6115 - val_accuracy: 0.2432 - val_precision_5: 0.3636 - val_recall_5: 0.0672
Epoch 3/30
542/542 [==============================] - 364s 671ms/step - loss: 0.6490 - accuracy: 0.3732 - precision_5: 0.4053 - recall_5: 0.2064 - val_loss: 0.6711 - val_accuracy: 0.2167 - val_precision_5: 0.3380 - val_recall_5: 0.2090
Epoch 4/30
542/542 [==============================] - 364s 671ms/step - loss: 0.6335 - accuracy: 0.3912 - precision_5: 0.4381 - recall_5: 0.2473 - val_loss: 0.6311 - val_accuracy: 0.2921 - val_precision_5: 0.3917 - val_recall_5: 0.4660
Epoch 5/30
542/542 [==============================] - 3

Salvar o melhor modelo com base em val_accuracy

In [ ]:
model.save("transformer_3_3", save_format="tf")

Análise do modelo

In [ ]:
history.history

{'loss': [35.34874725341797,
  0.7052193880081177,
  0.6489853858947754,
  0.6334955096244812,
  0.6197541952133179,
  0.6921444535255432,
  0.6844468116760254,
  0.8313703536987305,
  1.1552566289901733,
  0.8402384519577026,
  0.9503840208053589,
  1.0260976552963257,
  2.0222511291503906,
  0.608992338180542,
  0.7160966396331787,
  0.8955371379852295,
  0.6332851648330688,
  3.579631805419922,
  0.561310350894928,
  0.6631836891174316,
  0.6333733797073364,
  5.296660423278809,
  0.6525164842605591,
  0.5762068629264832,
  0.6243041753768921,
  0.6162014603614807,
  6.276973247528076,
  0.5914879441261292,
  0.5694247484207153],
 'accuracy': [0.29908162355422974,
  0.3770577013492584,
  0.3732455372810364,
  0.39115116000175476,
  0.39271068572998047,
  0.3724946677684784,
  0.35354936122894287,
  0.3340842127799988,
  0.3073413074016571,
  0.34863975644111633,
  0.34211286902427673,
  0.3022584319114685,
  0.3131750822067261,
  0.36458152532577515,
  0.3484664559364319,
  0.327326

In [ ]:
evaluation_results = model.evaluate(X_test, y_test)

68/68 [==============================] - 16s 240ms/step - loss: 0.7603 - accuracy: 0.4220 - precision_5: 0.2805 - recall_5: 0.5762


In [ ]:
y_pred = model.predict(X_test, verbose=False)

In [ ]:
y_pred

array([[0.5869944 , 0.5361448 , 0.05976122, 0.6242032 , 0.07294174],
       [0.5841311 , 0.5040231 , 0.0918911 , 0.5129537 , 0.06160607],
       [0.6016284 , 0.62375706, 0.0222728 , 0.81408453, 0.10655572],
       ...,
       [0.5904255 , 0.579936  , 0.0316939 , 0.76143336, 0.09235103],
       [0.59316933, 0.6024492 , 0.02349101, 0.81132746, 0.1030611 ],
       [0.5838784 , 0.501112  , 0.09548724, 0.5025709 , 0.06066054]],
      dtype=float32)

In [ ]:
y_test

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

In [ ]:
#define limiar(threshold) para converter as previsões contínuas em rótulos binários e poder usar o classification_report
threshold = 0.3

#aplicando o limiar às previsões
y_pred_ = (y_pred > threshold).astype(int)

print(classification_report(y_test, y_pred_))

              precision    recall  f1-score   support

           0       0.23      1.00      0.37       497
           1       0.12      1.00      0.22       262
           2       0.45      0.05      0.08       551
           3       0.46      0.98      0.63       963
           4       0.00      0.00      0.00       521

   micro avg       0.27      0.62      0.38      2794
   macro avg       0.25      0.60      0.26      2794
weighted avg       0.30      0.62      0.32      2794
 samples avg       0.27      0.65      0.37      2794



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.0037054191755442334


In [ ]:
#define limiar(threshold) para converter as previsões contínuas em rótulos binários e poder usar o classification_report
threshold = 0.5

#aplicando o limiar às previsões
y_pred_ = (y_pred > threshold).astype(int)

print(classification_report(y_test, y_pred_))

              precision    recall  f1-score   support

           0       0.23      1.00      0.37       497
           1       0.13      0.88      0.22       262
           2       0.50      0.03      0.06       551
           3       0.50      0.90      0.64       963
           4       0.00      0.00      0.00       521

   micro avg       0.28      0.58      0.38      2794
   macro avg       0.27      0.56      0.26      2794
weighted avg       0.32      0.58      0.32      2794
 samples avg       0.29      0.60      0.37      2794



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.026401111625752663


In [ ]:
#define limiar(threshold) para converter as previsões contínuas em rótulos binários e poder usar o classification_report
threshold = 0.75

#aplicando o limiar às previsões
y_pred_ = (y_pred > threshold).astype(int)

print(classification_report(y_test, y_pred_))

              precision    recall  f1-score   support

           0       0.22      0.01      0.02       497
           1       0.60      0.22      0.32       262
           2       0.33      0.00      0.01       551
           3       0.55      0.42      0.48       963
           4       0.00      0.00      0.00       521

   micro avg       0.55      0.17      0.26      2794
   macro avg       0.34      0.13      0.16      2794
weighted avg       0.35      0.17      0.20      2794
 samples avg       0.21      0.20      0.21      2794



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.1871236683649838


In [ ]:
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.32      0.42      0.37       497
           1       0.28      0.50      0.36       187
           2       0.19      0.02      0.03       317
           3       0.52      0.66      0.58       916
           4       0.00      0.00      0.00       242

    accuracy                           0.42      2159
   macro avg       0.26      0.32      0.27      2159
weighted avg       0.35      0.42      0.37      2159



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.1871236683649838
